In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tnguy70/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/tnguy70/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
df.describe();

In [5]:
df.head();

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['name']))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['cozy', 'entire', 'floor', 'of', 'brownstone']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['cozy', 'entire', 'floor', 'brownstone']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

/software/anaconda3/5.3.0/lib/python3.7/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [13]:
# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.205*"bedroom" + 0.063*"sunny" + 0.036*"train" + 0.034*"close" + '
  '0.032*"clean" + 0.029*"luxury" + 0.028*"min" + 0.024*"walk" + 0.023*"huge" '
  '+ 0.018*"share"'),
 (1,
  '0.183*"apartment" + 0.056*"heart" + 0.047*"home" + 0.034*"time" + '
  '0.034*"location" + 0.030*"village" + 0.026*"away" + 0.026*"good" + '
  '0.022*"place" + 0.022*"amazing"'),
 (2,
  '0.298*"room" + 0.275*"private" + 0.071*"bed" + 0.037*"large" + '
  '0.035*"bathroom" + 0.033*"comfortable" + 0.028*"side" + 0.018*"stay" + '
  '0.018*"park" + 0.014*"low"'),
 (3,
  '0.093*"spacious" + 0.077*"studio" + 0.062*"cozy" + 0.058*"modern" + '
  '0.057*"beautiful" + 0.034*"bright" + 0.032*"new" + 0.030*"renovate" + '
  '0.029*"quiet" + 0.028*"view"')]


In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.521068727981805

Coherence Score:  0.27562559996938696


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/software/anaconda3/5.3.0/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.407223  0.008820       1        1  25.836304
3     -0.142157 -0.355127       2        1  24.926939
0     -0.098489  0.053042       3        1  24.697014
1     -0.166577  0.293266       4        1  24.539743, topic_info=     Category           Freq         Term          Total  loglift  logprob
term                                                                      
10    Default  209651.000000         room  209651.000000  30.0000  30.0000
60    Default  193295.000000      private  193295.000000  29.0000  29.0000
24    Default  137641.000000      bedroom  137641.000000  28.0000  28.0000
7     Default  121956.000000    apartment  121956.000000  27.0000  27.0000
5     Default   63365.000000     spacious   63365.000000  26.0000  26.0000
6     Default   52439.000000       studio   52439.000000  25.0000  25.0000
40    Default   49894.000000          bed   49894.000000  24.0000  24.0000
38    Default   42480.000000        sunny   42480.000000  23.0000  23.0000
1     Default   42161.000000         cozy   42161.000000  22.0000  22.0000
76    Default   39174.000000       modern   39174.000000  21.0000  21.0000
55    Default   38768.000000    beautiful   38768.000000  20.0000  20.0000
73    Default   37557.000000        heart   37557.000000  19.0000  19.0000
51    Default   31302.000000         home   31302.000000  18.0000  18.0000
8     Default   25872.000000        large   25872.000000  17.0000  17.0000
224   Default   23898.000000        train   23898.000000  16.0000  16.0000
85    Default   24501.000000     bathroom   24501.000000  15.0000  15.0000
62    Default   23397.000000       bright   23397.000000  14.0000  14.0000
102   Default   22986.000000         time   22986.000000  13.0000  13.0000
66    Default   22772.000000     location   22772.000000  12.0000  12.0000
178   Default   22639.000000        close   22639.000000  11.0000  11.0000
70    Default   23134.000000  comfortable   23134.000000  10.0000  10.0000
168   Default   21897.000000          new   21897.000000   9.0000   9.0000
108   Default   21274.000000        clean   21274.000000   8.0000   8.0000
69    Default   20150.000000      village   20150.000000   7.0000   7.0000
206   Default   20187.000000     renovate   20187.000000   6.0000   6.0000
101   Default   19723.000000        quiet   19723.000000   5.0000   5.0000
106   Default   19526.000000       luxury   19526.000000   4.0000   4.0000
18    Default   19748.000000         side   19748.000000   3.0000   3.0000
151   Default   18816.000000          min   18816.000000   2.0000   2.0000
112   Default   18862.000000         view   18862.000000   1.0000   1.0000
...       ...            ...          ...            ...      ...      ...
73     Topic4   37557.156250        heart   37557.933594   1.4049  -2.8786
51     Topic4   31301.730469         home   31302.505859   1.4049  -3.0608
102    Topic4   22985.693359         time   22986.470703   1.4048  -3.3696
66     Topic4   22771.833984     location   22772.623047   1.4048  -3.3790
69     Topic4   20150.197266      village   20150.984375   1.4048  -3.5013
200    Topic4   17133.410156         away   17134.191406   1.4048  -3.6634
13     Topic4   17121.125000         good   17121.906250   1.4048  -3.6642
221    Topic4   14569.197266        place   14569.979492   1.4048  -3.8256
249    Topic4   14423.022461      amazing   14423.803711   1.4048  -3.8357
86     Topic4   12933.507812        queen   12934.315430   1.4048  -3.9447
22     Topic4   10191.300781       lovely   10192.089844   1.4048  -4.1829
250    Topic4    9099.993164        oasis    9100.777344   1.4048  -4.2962
333    Topic4    8982.637695         step    8983.426758   1.4048  -4.3092
135    Topic4    9111.526367    luxurious    9112.341797   1.4048  -4.2949
190    Topic4    8234.023438         cute    8234.819336   1.4048  -4.3962
119    Topic4    7831.835449     